<a href="https://colab.research.google.com/github/AnantaDey/Rice_Field_Detection/blob/main/Sentinel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 125.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 92.0 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-non

In [2]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ENERXppIUJaNOf3VqRmgo_kwABtBuT3ZNEfI-RnO7wQ&tc=45noAEkKHTZdd62mAHO15UMxcjCBn48DkP1eu6U8pTA&cc=2cA1z6ttsLvgDoGWeauf1i4s1a1TZOMRR3QUQVsOpFY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VPDF2jwygUHEWAb1vRBFnJ9Idzy43V7PqIkgRSAbXvX63gFIsysZH4

Successfully saved authorization token.


In [3]:
import geemap
from geemap.plot import center_zoom_to_xy_range
import ipywidgets as widgets
from ipyleaflet import WidgetControl
from geemap import geojson_to_ee

Map = geemap.Map(center =[23.8402, 87.6186], zoom_start=9)

dc = Map.draw_control

roi = []
# Handle draw events
def handle_draw(self, action, geo_json):
    geometry = geo_json['geometry']
    if geometry['type'] == 'Polygon':
        coordinates = geometry['coordinates'][0]
        roi.append(coordinates)
        print("Polygon coordinates:", coordinates)
dc.on_draw(handle_draw)
Map

Map(center=[23.8402, 87.6186], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

Polygon coordinates: [[87.884819, 24.249469], [87.758494, 24.256981], [87.665124, 24.226929], [87.74751, 24.181836], [87.706317, 24.124195], [87.623931, 24.154272], [87.626677, 24.094111], [87.569007, 24.043956], [87.503102, 23.993781], [87.420717, 23.943586], [87.349316, 23.983743], [87.310869, 23.986253], [87.3246, 23.91597], [87.30263, 23.833089], [87.233976, 23.797911], [87.162575, 23.785345], [87.074696, 23.782831], [86.986818, 23.802937], [86.907179, 23.772778], [86.84127, 23.694835], [86.893449, 23.601745], [86.995058, 23.533773], [87.044488, 23.433009], [87.140605, 23.392682], [87.277914, 23.362429], [87.401493, 23.359907], [87.687085, 23.309469], [87.950719, 23.266581], [88.208513, 23.261534], [88.318361, 23.493477], [88.538056, 23.594194], [88.587488, 23.730041], [88.560026, 23.835601], [88.439193, 23.981234], [88.488625, 24.136728], [88.422716, 24.241956], [88.345823, 24.342092], [88.208513, 24.437148], [88.087681, 24.512141], [87.955863, 24.557116], [87.922909, 24.47715], [

In [72]:
##Drawaing the Map
Map = geemap.Map(center =[23.8402, 87.6186], zoom_start=10)

##Defining the area of interest
polygon = ee.Geometry.Polygon(roi)


#Collecting the images
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2021-06-01', '2021-06-15').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
sentinel2 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2021-06-16', '2021-06-30').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
sentinel3 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2021-07-01', '2021-07-15').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
sentinel4 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2021-07-16', '2021-07-31').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
sentinel5 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2021-08-01', '2021-08-15').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
sentinel6 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2021-08-16', '2021-08-31').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
sentinel7 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2021-09-01', '2021-09-15').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
sentinel8 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2021-09-16', '2021-09-30').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
sentinel9 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2021-10-01', '2021-10-15').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
sentinel10 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2021-10-16', '2021-10-31').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
"""
sentinel11 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2022-11-16', '2022-11-30').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
sentinel12 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2022-12-01', '2022-12-15').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
sentinel13 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2022-12-16', '2022-12-31').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
sentinel14 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2023-01-01', '2023-01-15').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
sentinel15 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2023-04-11', '2023-04-25').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.Or(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'), ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))).filterBounds(polygon)
"""



##Making the bands
image1 = sentinel1.select('VH').mean().rename('VH1')
image2 = sentinel2.select('VH').mean().rename('VH2')
image3 = sentinel3.select('VH').mean().rename('VH3')
image4 = sentinel4.select('VH').mean().rename('VH4')
image5 = sentinel5.select('VH').mean().rename('VH5')
image6 = sentinel6.select('VH').mean().rename('VH6')
image7 = sentinel7.select('VH').mean().rename('VH7')
image8 = sentinel8.select('VH').mean().rename('VH8')
image9 = sentinel9.select('VH').mean().rename('VH9')
image10 = sentinel10.select('VH').mean().rename('VH10')
"""
image11 = sentinel10.select('VH').mean().rename('VH11')
image12 = sentinel10.select('VH').mean().rename('VH12')
image13 = sentinel10.select('VH').mean().rename('VH13')
image14 = sentinel10.select('VH').mean().rename('VH14')
image15 = sentinel10.select('VH').mean().rename('VH15')
"""

stacked = image1.addBands([image2,image3,image4,image5,image6,image7,image8,image9,image10]).clip(polygon)

stacked_scaled = stacked.multiply(10).add(350).uint8();
bands = ['VH2', 'VH4', 'VH9'];
display = {'bands': bands, 'min': 0, 'max': 220};

Map.addLayer(stacked_scaled, display, 'stacked')

# Get the DrawControl
dc = Map.draw_control

# List of recognised Fields
polygon_coordinates = []

# Handle draw events
def handle_draw(self, action, geo_json):
    geometry = geo_json['geometry']
    if geometry['type'] == 'Polygon':
        coordinates = geometry['coordinates'][0]
        polygon_coordinates.append(coordinates)
        print("Polygon coordinates:", coordinates)

dc.on_draw(handle_draw)


Map

Map(center=[23.8402, 87.6186], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

Polygon coordinates: [[87.162562, 23.564813], [87.16179, 23.561666], [87.165265, 23.561234], [87.166338, 23.564184], [87.162562, 23.564813]]
Polygon coordinates: [[87.411495, 23.845552], [87.41156, 23.842843], [87.415421, 23.843177], [87.41555, 23.84565], [87.411495, 23.845552]]
Polygon coordinates: [[87.400784, 23.829713], [87.400526, 23.825394], [87.40662, 23.825473], [87.406448, 23.828849], [87.400784, 23.829713]]
Polygon coordinates: [[88.179484, 24.452853], [88.178669, 24.447462], [88.184676, 24.446056], [88.185534, 24.450158], [88.179484, 24.452853]]


## For trainning Dataset

In [73]:
# function to get the rice fields of requirement
def generate_fields (polygon_coordinates):
  fields = []
  for field in polygon_coordinates :
    field = np.array(field)
    field = np.transpose(field)
    for i in range(10):
      coeff = np.random.rand(field.shape[1])
      coeff /= coeff.sum()
      fieldpoint = np.dot(field, coeff)
      fields.append(list(fieldpoint))
  return fields

In [74]:
# Getting the rice fields
fields = generate_fields(polygon_coordinates) #Here the polygon_coordinates contains the primary rice fields
fields

[[87.16360041806321, 23.562765837296194],
 [87.16315649838603, 23.563532728098153],
 [87.16410393633944, 23.563136008472302],
 [87.16439380541566, 23.5640139466442],
 [87.164263913782, 23.563856641119543],
 [87.16366258333775, 23.563337379338897],
 [87.16495127943804, 23.563028460783208],
 [87.16353444403167, 23.56278950754699],
 [87.16414803027621, 23.56378619828246],
 [87.16259753264974, 23.56332129106667],
 [87.41255137404251, 23.8445153877861],
 [87.41241614146763, 23.84464690164785],
 [87.4130791392918, 23.84432821738736],
 [87.4131105869862, 23.84463585512272],
 [87.4130496616197, 23.845306767778464],
 [87.41253348160258, 23.844614292289943],
 [87.41380237724287, 23.844786494064348],
 [87.41325866825937, 23.844006696888805],
 [87.41426211878081, 23.844831578411608],
 [87.41298008608534, 23.844434838980533],
 [87.40165360216861, 23.82776075785228],
 [87.40340489565097, 23.827353737357658],
 [87.40316912600733, 23.828557710200975],
 [87.40204141048463, 23.827599135903203],
 [87.402

In [75]:
# Making the dataframe which will contain our training dataset
df = pd.DataFrame(np.array(fields)).rename({0 : 'latitude', 1 : 'longitude'}, axis = 1)
df.shape


(40, 2)

In [76]:
# Adding Rice-Groups Class
df['Rice-Groups']=None

# Columns
column=['latitude', 'longitude', 'Rice-Groups','VH1', 'VH2', 'VH3', 'VH4',
       'VH5', 'VH6', 'VH7', 'VH8', 'VH9', 'VH10']

In [77]:

df.shape

(40, 3)

In [78]:
for i in range(1,11):
  ls = []
  for j in range(df.shape[0]):
    pointOfInterest = ee.Geometry.Point([df.iloc[j][0],df.iloc[j][1]])
    bandValues = stacked_scaled.reduceRegion(
      reducer = ee.Reducer.first(),  # You can choose a different reducer if needed
      geometry = pointOfInterest,
      scale = 30,  # Specify the scale/resolution for the analysis
      maxPixels = 30  # Set a limit for the number of pixels to be processed
    )
    ls.append(ee.Number(bandValues.get('VH'+str(i))).toInt().getInfo())
  df['VH'+str(i)] = pd.Series(np.array(ls))
  df['Rice-Groups']='Water'


In [79]:
df

,latitude,longitude,Rice-Groups,VH1,VH2,VH3,VH4,VH5,VH6,VH7,VH8,VH9,VH10
0,87.163600,23.562766,Water,109,82,129,80,100,82,101,63,82,80
1,87.163156,23.563533,Water,72,83,98,80,84,78,65,75,88,93
2,87.164104,23.563136,Water,88,103,97,81,78,81,69,84,84,70
3,87.164394,23.564014,Water,88,79,78,69,73,77,81,65,68,80
4,87.164264,23.563857,Water,88,79,78,69,73,77,81,65,68,80
5,87.163663,23.563337,Water,84,81,80,97,91,87,77,69,91,73
6,87.164951,23.563028,Water,80,93,84,79,90,61,66,83,70,81
7,87.163534,23.562790,Water,109,82,129,80,100,82,101,63,82,80
8,87.164148,23.563786,Water,90,73,78,89,93,58,72,72,69,88
9,87.162598,23.563321,Water,77,91,64,80,75,87,79,72,80,69


In [80]:
# Concatenate the DataFrames vertically
df1 = pd.concat([df1,df])

# # Reset the index of the concatenated DataFrame
df1.reset_index(drop=True, inplace=True)

In [81]:
df1.shape

(280, 13)

In [82]:
df1

,latitude,longitude,Rice-Groups,VH1,VH2,VH3,VH4,VH5,VH6,VH7,VH8,VH9,VH10
0,87.529139,23.679022,Class1,186,92,119,79,122,122,145,182,202,192
1,87.529187,23.679031,Class1,137,78,94,78,105,120,159,177,173,191
2,87.529136,23.679004,Class1,186,92,119,79,122,122,145,182,202,192
3,87.529173,23.679032,Class1,137,78,94,78,105,120,159,177,173,191
4,87.529208,23.679028,Class1,137,78,94,78,105,120,159,177,173,191
...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,88.181084,24.450322,Water,69,88,88,69,86,61,73,74,62,63
276,88.181883,24.450106,Water,61,64,69,55,53,55,78,80,69,74
277,88.181182,24.449193,Water,65,65,68,71,56,77,76,68,71,56
278,88.181438,24.450163,Water,37,74,66,75,64,55,80,59,69,78


In [83]:
df1.to_csv("test.csv")